In [2]:
import numpy as np 
import pandas as pd 
import nltk 
import sklearn 
from sklearn import*
import re 
from tqdm import tqdm 
import catboost as ctb
import joblib

In [3]:
tqdm.pandas()

In [4]:
#грузим датасеты

In [14]:
df = pd.read_csv('clean_df.csv')
df = df.dropna()

In [7]:
def hatred_upper(text):
    upper = len([i for i in text if i.isupper()])
    lower = len([i for i in text if i.islower()])
    return upper/(upper+lower)

def hatred_punc(text):
    punct = len(re.findall(r'[^\w\s]',text))
    text = len(re.findall(r'\w+',text))
    return punct/text


In [15]:
idf = feature_extraction.text.TfidfVectorizer(max_features=2000,ngram_range=(1,2))

In [16]:
idf_text=idf.fit_transform(df.clean_text).toarray()

In [17]:
#Создаем фичи для улучшения работы модели
df['upper']=df.ttext.apply(hatred_upper)
df['punct']=df.ttext.apply(hatred_punc)
df['title']=df.ttext.apply(lambda x: len([i for i in x if i.istitle()==True]))
df['pos_sc'] = df.ttext.apply(lambda text: len(re.findall(r'\)|D',text)))
df['neg_sc'] = df.ttext.apply(lambda text: len(re.findall(r'\(|C|c|С|c',text)))


In [18]:
#проверяем силу фичей
df.corr()

,ttype,upper,punct,title,pos_sc,neg_sc
ttype,1.000000,0.063915,-0.003387,0.089728,0.533223,-0.347500
upper,0.063915,1.000000,0.139611,0.913370,0.076962,0.189603
punct,-0.003387,0.139611,1.000000,0.066682,0.214800,0.550125
title,0.089728,0.913370,0.066682,1.000000,0.086383,0.167231
pos_sc,0.533223,0.076962,0.214800,0.086383,1.000000,-0.180788
neg_sc,-0.347500,0.189603,0.550125,0.167231,-0.180788,1.000000


In [19]:
df

,ttext,ttype,preprocessed,clean_text,upper,punct,title,pos_sc,neg_sc
0,"@first_timee хоть я и школота, но поверь, у на...",1,"['хот', 'я', 'и', 'школот', 'но', 'повер', 'у'...",хот я и школот но повер у нас то же сам общест...,0.012987,0.294118,1,2,0
1,"Да, все-таки он немного похож на него. Но мой ...",1,"['да', 'все', 'так', 'он', 'немн', 'похож', 'н...",да все так он немн похож на нег но мо мальчик ...,0.054545,0.266667,3,1,0
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,"['ну', 'ты', 'идиотк', 'я', 'испуга', 'за', 'т...",ну ты идиотк я испуга за теб,0.128205,0.666667,5,1,1
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,"['кто', 'то', 'в', 'угл', 'сид', 'и', 'погиба'...",кто то в угл сид и погиба от голод а мы ещ пор...,0.084211,0.448276,8,2,1
4,@irina_dyshkant Вот что значит страшилка :D\nН...,1,"['вот', 'что', 'знач', 'страшилк', 'но', 'блин...",вот что знач страшилк но блин посмотр все част...,0.039604,0.333333,4,2,0
...,...,...,...,...,...,...,...,...,...
226829,Но не каждый хочет что то исправлять:( http://...,-1,"['но', 'не', 'кажд', 'хочет', 'что', 'то', 'ис...",но не кажд хочет что то исправля,0.173913,0.636364,8,2,2
226830,скучаю так :-( только @taaannyaaa вправляет мо...,-1,"['скуча', 'так', 'тольк', 'вправля', 'мозг', '...",скуча так тольк вправля мозг но я все равн скуча,0.000000,0.454545,0,0,1
226831,"Вот и в школу, в говно это идти уже надо(",-1,"['вот', 'и', 'в', 'школ', 'в', 'говн', 'эт', '...",вот и в школ в говн эт идт уж над,0.033333,0.200000,1,0,1
226832,"RT @_Them__: @LisaBeroud Тауриэль, не грусти :...",-1,"['тауриэл', 'не', 'груст', 'обня']",тауриэл не груст обня,0.162162,1.142857,6,0,1


In [20]:
end_df=np.hstack([idf_text,df.iloc[:,-5:].values])

In [21]:
end_df.shape

(226830, 2005)

In [22]:
#Выбираем классификатор
clf_mnb =naive_bayes.MultinomialNB(alpha=10)

In [23]:
#Обучаем модель
clf_mnb.fit(end_df,df['ttype'])

MultinomialNB(alpha=10)

In [24]:
# Проверяем эффективность по метрике
np.mean(model_selection.cross_val_score(clf_mnb,end_df,df.ttype,cv=5,scoring='roc_auc',verbose=5))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV]  ................................................................
[CV] .................................... , score=0.986, total=   4.7s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.9s remaining:    0.0s
[CV]  ................................................................
[CV] .................................... , score=0.987, total=   3.1s
[CV]  ................................................................
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    8.3s remaining:    0.0s
[CV] .................................... , score=0.986, total=   3.3s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   11.9s remaining:    0.0s
[CV]  ................................................................
[CV] .................................... , score=0.985, total=   3.0s
[CV]  ................................................................
[Parallel(n_jobs=1)]: Done   4 out of   

0.9868535213546197

In [25]:
def nlp_ex(text):
    # some feature engineering
    upper = [[hatred_upper(text)]]
    punctuation = [[hatred_punc(text)]]
    title = [[len([i for i in text if i.istitle()==True])]]
    pos_sc = [[len(re.findall(r'\)|D',text))]]
    neg_sc = [[len(re.findall(r'\(|C|c|С|c',text))]]
    #text preprocessing
    stemmer = nltk.stem.SnowballStemmer('russian')
    preproc=[stemmer.stem(i) for i in nltk.regexp_tokenize(text.lower(),r'[А-я]+')]
    text = ' '.join(preproc)
    idf_text= (idf.transform([text])).toarray()
    # concat type of file that we needeed
    end_text=np.hstack([idf_text,upper,punctuation,title,pos_sc,neg_sc])
    # use model
    pred = clf_mnb.predict_proba(end_text)

    return print(f" Negative {pred[0][0]:.2%}, Positive {pred[0][1]:.2%}")

In [26]:
for i in ['эт хорош','позитив',"понравил вечер"," эт прост моя любов","милота"]:
    nlp_ex(i)

Negative 37.05%, Positive 62.95%
 Negative 49.34%, Positive 50.66%
 Negative 30.67%, Positive 69.33%
 Negative 47.14%, Positive 52.86%
 Negative 49.34%, Positive 50.66%


In [29]:
for i in ['ужас','ну что за день(',"ненавижу тебя","простo тварь","это просто фиаско!"]:
    nlp_ex(i)

Negative 82.39%, Positive 17.61%
 Negative 92.15%, Positive 7.85%
 Negative 80.94%, Positive 19.06%
 Negative 50.57%, Positive 49.43%
 Negative 48.13%, Positive 51.87%
